Playground
==

You can test the model here

In [3]:
# build & run test
!./scripts/build_n_test.sh

-- pybind11 v2.9.1 
-- Configuring done
-- Generating done
-- Build files have been written to: /home/nfs_home/blender/repos/DummyML/build
[100%] Built target dummyml
============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-7.1.1, pluggy-1.0.0
rootdir: /home/nfs_home/blender/repos/DummyML, configfile: pytest.ini, testpaths: tests
plugins: typeguard-2.13.3
collected 5 items                                                              

tests/test_dummy_classifier.py ..                                        [ 40%]
tests/test_dummy_regression.py .                                         [ 60%]
tests/test_naive_bayes_classifier.py ..                                  [100%]

============================== 5 passed in 1.27s ===============================


In [1]:
# Before playing, make sure you already build the project
import dummyml
import numpy as np

In [3]:
dummyX = np.zeros((4,))
nb_classifier = dummyml.naive_bayes_classifier(0,0)

nb_classifier.load('model/myIRIS.dmy')
print("Result:",nb_classifier(dummyX))

Result: [4.27880778e-82 2.10843320e-01 7.89156680e-01]
[WARNING] naive_bayes_classifier Ctr: _class_number can't be zero, forced changed to 1.
